In [ ]:
from pathlib import Path
import time
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
from stochopy.optimize import minimize
from tqdm import tqdm
import similaritymeasures
import random
import matplotlib.pyplot as plt
from shapely.ops import Point, MultiLineString, LineString
from importlib import reload
import datetime
from scipy.spatial.distance import directed_hausdorff
np.set_printoptions(suppress=True)
from matplotlib.ticker import MultipleLocator

from bikewaysim.paths import config, stadia_toner, maptiler_streets
from bikewaysim.impedance_calibration import stochastic_optimization, speedfactor
from bikewaysim.network import modeling_turns
from bikewaysim.routing import rustworkx_routing_funcs
from bikewaysim.impedance_calibration import optimization_viz, utils, post_calibration, impedance_functions, loss_functions

In [ ]:
with (config['calibration_fp'] / 'results/bootstrap_final,validation,0.pkl').open('rb') as fh:
    bootstrap_model = pickle.load(fh)

links, turns, length_dict, geo_dict, turn_G = rustworkx_routing_funcs.import_calibration_network(config)

In [ ]:
from importlib import reload
reload(stochastic_optimization)
 
# links['link_cost']

# run impedance routing on all the provided ODs
base_impedance_col = "travel_time_min"
rustworkx_routing_funcs.back_to_base_impedance(base_impedance_col,links,turns,turn_G)

betas_tup = bootstrap_model['betas_tup']
betas = [x['beta'] for x in betas_tup] # get betas

#update impedances
print(betas)
_ = rustworkx_routing_funcs.impedance_update(
    betas,betas_tup,
    impedance_functions.link_impedance_function,
    base_impedance_col,
    None,
    impedance_functions.turn_impedance_function,
    links,turns,turn_G)

In [ ]:
highest_multiplier = links.groupby('linkid')['multiplier'].apply(lambda x: x.abs().idxmax()).tolist()
highest_multiplier = links.loc[highest_multiplier]
highest_multiplier['pct'] = highest_multiplier['multiplier'] * 100
highest_multiplier[['travel_time_min','link_cost','multiplier','pct']]

In [ ]:
highest_multiplier.loc[highest_multiplier['multiplier'] > 0,['travel_time_min','link_cost','multiplier','pct','geometry']].to_file(config['calibration_fp']/"network_impedance_change2.gpkg",layer='increased_impedance')
highest_multiplier.loc[highest_multiplier['multiplier'] < 0,['travel_time_min','link_cost','multiplier','pct','geometry']].to_file(config['calibration_fp']/"network_impedance_change2.gpkg",layer='decreased_impedance')
highest_multiplier.loc[highest_multiplier['multiplier'] == 0,['travel_time_min','link_cost','multiplier','pct','geometry']].to_file(config['calibration_fp']/"network_impedance_change2.gpkg",layer='base_case')

## For the table 

In [ ]:
cols = ['3+lpd','(30,inf) mph','[4,6) grade','[6,inf) grade','bike lane','multi use path']
beta_vec = [x['beta'] for x in betas_tup if x['col'] in cols]
beta_vec

In [ ]:
x = np.array([
    [0,0,0,0,0,0],
    [0,0,1,0,0,0],
    [0,0,0,1,0,0],
    [0,0,0,0,0,1],
    [0,0,1,0,0,1],
    [0,0,0,1,0,1],
    [0,0,0,0,0,0],
    [0,0,1,0,0,0],
    [0,0,0,1,0,0],
    [1,0,0,0,0,0],
    [1,0,1,0,0,0],
    [1,0,0,1,0,0],
    [1,1,0,0,0,0],
    [1,1,1,0,0,0],
    [1,1,0,1,0,0],
    [0,1,0,0,0,0],
    [0,1,1,0,0,0],
    [0,1,0,1,0,0],
    [0,0,0,0,1,0],
    [0,0,1,0,1,0],
    [0,0,0,1,1,0],
    [1,0,0,0,1,0],
    [1,0,1,0,1,0],
    [1,0,0,1,1,0],
    [1,1,0,0,1,0],
    [1,1,1,0,1,0],
    [1,1,0,1,1,0],
    [0,1,0,0,1,0],
    [0,1,1,0,1,0],
    [0,1,0,1,1,0]
])

In [ ]:
stats = np.sum(x * beta_vec,axis=1).round(2)
stats

## Basic Stats

In [ ]:
stats

In [ ]:
print(stats.max())
print(stats.min())